#Compute molecular descriptors and perform EDA
 A follow along project on drug discovery for *Zaire ebolavirus*
 

by Rea Kalampaliki

**Digital mentor**: Chanin Nantasenamat ([Data Professor](https://www.youtube.com/watch?v=qWVTxfLq2ak&list=PLtqF5YXg7GLlQJUv9XJ3RWdd5VYGwBHrP&index=2))

##Install Conda and rdkit

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-02-01 17:03:41--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  78.9MB/s    in 1.0s    

2021-02-01 17:03:42 (78.9 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6

##Load bioactivity data

In [ ]:
import pandas as pd
df = pd.read_csv('/content/GAR_transformylase_03_bioactivity_data_curated.csv')
print(df.shape)
df.head()

(69, 4)


,molecule_chembl_id,canonical_smiles,IC50,class
0,CHEMBL289923,NCCCC(NC(=O)c1ccc(NCC2CNc3nc(N)nc(O)c3C2)cc1)C...,4800.0,intermediate
1,CHEMBL152172,Nc1nc(N)c(CCCN(C=O)c2ccc(C(=O)NC(CCC(=O)O)C(=O...,18000.0,inactive
2,CHEMBL153550,CN(CCCc1c(N)nc(N)nc1O)c1ccc(C(=O)NC(CCC(=O)O)C...,4300.0,intermediate
3,CHEMBL13659,Nc1nc(N)c(CCCNc2ccc(C(=O)N[C@@H](CCC(=O)O)C(=O...,630.0,active
4,CHEMBL158034,Nc1nc(O)c2c(n1)NCC1CCN(c3ccc(C(=O)NC(CCC(=O)O)...,20000.0,inactive


##Calculate Lipinski descriptors

Rule-of-five describes a drug as following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10 

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
    
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
df_lipinski = lipinski(df.canonical_smiles)

print(df_lipinski.shape)
df_lipinski.head()

(69, 4)


,MW,LogP,NumHDonors,NumHAcceptors
0,429.481,0.3826,7.0,9.0
1,460.447,-0.0100,6.0,9.0
2,446.464,0.4634,6.0,9.0
3,432.437,0.4391,7.0,9.0
4,470.486,0.8478,6.0,9.0


In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)

print(df_combined.shape)
df_combined.head()

(69, 8)


,molecule_chembl_id,canonical_smiles,IC50,class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL289923,NCCCC(NC(=O)c1ccc(NCC2CNc3nc(N)nc(O)c3C2)cc1)C...,4800.0,intermediate,429.481,0.3826,7.0,9.0
1,CHEMBL152172,Nc1nc(N)c(CCCN(C=O)c2ccc(C(=O)NC(CCC(=O)O)C(=O...,18000.0,inactive,460.447,-0.0100,6.0,9.0
2,CHEMBL153550,CN(CCCc1c(N)nc(N)nc1O)c1ccc(C(=O)NC(CCC(=O)O)C...,4300.0,intermediate,446.464,0.4634,6.0,9.0
3,CHEMBL13659,Nc1nc(N)c(CCCNc2ccc(C(=O)N[C@@H](CCC(=O)O)C(=O...,630.0,active,432.437,0.4391,7.0,9.0
4,CHEMBL158034,Nc1nc(O)c2c(n1)NCC1CCN(c3ccc(C(=O)NC(CCC(=O)O)...,20000.0,inactive,470.486,0.8478,6.0,9.0


##Convert IC50 to pIC50

To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially **-log10(IC50)**.

In [ ]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['IC50_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('IC50_norm', 1)
        
    return x

IC50 values that are higher than 100,000,000, will give a negative value after apply the negative logarithmic transformation. To prevent that, before appyling `IC50` function, we will convert all IC50 values in `final_df` that higher than 100,000,000 to the maximum value 100,000,000.

In [ ]:
def norm_value(input):
    norm = []

    for i in input['IC50']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['IC50_norm'] = norm
    x = input.drop('IC50', 1)
        
    return x

In [ ]:
df_norm = norm_value(df_combined)
df_final = pIC50(df_norm)
print(df_final.shape)
df_final.head()

(69, 8)


,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL289923,NCCCC(NC(=O)c1ccc(NCC2CNc3nc(N)nc(O)c3C2)cc1)C...,intermediate,429.481,0.3826,7.0,9.0,5.318759
1,CHEMBL152172,Nc1nc(N)c(CCCN(C=O)c2ccc(C(=O)NC(CCC(=O)O)C(=O...,inactive,460.447,-0.0100,6.0,9.0,4.744727
2,CHEMBL153550,CN(CCCc1c(N)nc(N)nc1O)c1ccc(C(=O)NC(CCC(=O)O)C...,intermediate,446.464,0.4634,6.0,9.0,5.366532
3,CHEMBL13659,Nc1nc(N)c(CCCNc2ccc(C(=O)N[C@@H](CCC(=O)O)C(=O...,active,432.437,0.4391,7.0,9.0,6.200659
4,CHEMBL158034,Nc1nc(O)c2c(n1)NCC1CCN(c3ccc(C(=O)NC(CCC(=O)O)...,inactive,470.486,0.8478,6.0,9.0,4.698970


Save `df_final` to a CSV file

In [ ]:
df_final.to_csv("GAR_transformylase_04_bioactivity_data_3class_pIC50.csv", index = False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,2, figsize=(10,4))
sns.boxplot(data= df_combined, x= df_combined['IC50'], ax = ax[0])
sns.boxplot(data= df_final, x= df_final['pIC50'], ax = ax[1])
ax[0].set_xlabel('IC50', size = 14)
ax[1].set_xlabel('pIC50', size = 14)

##Remove the 'intermediate' bioactivy class

In [ ]:
final_two_classes = df_final[df_final['class'] != 'intermediate']

print(final_two_classes.shape)
final_two_classes.head()

##Exploratory Data Analysis (Chemical Space Analysis) via Lipinski descriptors

###Frequency plot of the two bioactivity classes (active - inactive)

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='class', data=final_two_classes, edgecolor='black')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.pdf')

###Scatterplot of the MW verus LogP

In [ ]:
plt.figure(figsize=(6, 6))

sns.scatterplot(x='MW', y='LogP', data=final_two_classes, hue='class', size='pIC50', alpha=0.7)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

Noticably, the active compounds have higher LogP (solubility) and higher pIC50 than the inactive compounds.

###Mann-Whitney U test function

In [ ]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'class']
  df = final_two_classes[selection]
  active = df[df['class'] == 'active']
  active = active[descriptor]

  selection = [descriptor, 'class']
  df = final_two_classes[selection]
  inactive = df[df['class'] == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'
  
  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

### pIC50 of active and inactive compounds (boxplot and MannWhitney U test)

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'pIC50', data = final_two_classes)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')

By definition, pIC50 value is lower than 5 for the class of inactive compounds and higher than 6 for the class of active compounds.

With further notice, inside the class of inactive compounds the pIC50 values are densly distributed between 4 and 5, while inside the class of active compounds pIC50 values are more widely distributed between 6 and 9.

Inside the class of active compounds some outlier pIC50 values around 11.5, 12 and 13 are also noticable. 

In [ ]:
mannwhitney('pIC50')

### MW of active and inactive classes (boxplot and MannWhitney U test)

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'MW', data = final_two_classes)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

In [ ]:
mannwhitney('MW')

### logP of active and inactive compounds (boxplot and MannWhitney U test)

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'LogP', data = final_two_classes)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')

In [ ]:
mannwhitney('LogP')

###NumHDonors of active and inactive compounds (boxplot and MannWhitney U test)

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'NumHDonors', data = final_two_classes)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.pdf')

In [ ]:
mannwhitney('NumHDonors')

###NumHAcceptors of active and inactive compounds (boxplot and MannWhitney U test)

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'NumHAcceptors', data = final_two_classes)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.pdf')

In [ ]:
mannwhitney('NumHAcceptors')

###Notes on the results of the statistical tests

**pIC50** values between the **active** and **inactive** compounds displayed s*tatistically significant difference*, which was expected because of the way the two classes of compounds where defined `(IC50 < 1,000 nM = Actives while IC50 > 10,000 nM = Inactives`, corresponding to `pIC50 > 6 = Actives and pIC50 < 5 = Inactives`).

Out of the four Lipinski molecular descriptors (LogP, MW, NumHDonors,NumHAccptors), the two, **LogP** and **NumHAcceptors**, exhibited *statistically significant difference* between the **active** and the **inactive** compounds.

##Zip the resulting files

In [ ]:
! zip -r results.zip . -i *.csv *.pdf